<a href="https://colab.research.google.com/github/enternalsaga/stablediffusion_webui/blob/master/SD_webui_colab_AIDVN_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, subprocess, time, glob

Model_from_URL = ''
Save_a_copy_in_Google_Drive = False


Extensions_from_URL = ''



# install A1111 in Colab
root = '/content/'
# make necessary directories if not exists
!mkdir -p {output_path}/outputs
!mkdir -p {output_path}/models
!mkdir -p {output_path}/hypernetworks



def clear():
    from IPython.display import clear_output; return clear_output()

def fetch_bytes(url_or_path):
    if str(url_or_path).startswith('http://') or str(url_or_path).startswith('https://'):
        from urllib.request import urlopen
        return urlopen(url_or_path)
    return open(url_or_path, 'r')

def packages():
    import sys, subprocess
    return [r.decode().split('==')[0] for r in subprocess.check_output([sys.executable, '-m', 'pip', 'freeze']).split()]

def downloadModel(url):
  if 'huggingface.co' in url:
    filename = url.split('/')[-1]
    filename = filename.removesuffix('?download=true')
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}  -o {filename}
  else:
    # civitai
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}

def download_models():
  %cd {root}/stable-diffusion-webui/models/Stable-diffusion
  print('⏳ Downloading models ...')
  if landscapeBING_model:
    downloadModel('https://huggingface.co/enternalsaga/LandscapeBING_v1/resolve/main/landscapebing_v1.safetensors')
  if minustypeUrbandesign_model:
    downloadModel('https://huggingface.co/enternalsaga/minustypeUrbandesign_v6-fp16-no-ema/resolve/main/minustypeUrbandesign_v6-fp16-no-ema.safetensors')
  if Photon_model:
    downloadModel('https://huggingface.co/digiplay/Photon_v1/resolve/main/photon_v1.safetensors')
  if epiCPhotoGasm_model:
    downloadModel('https://huggingface.co/jzli/epiCPhotoGasm-last-unicorn/resolve/main/epicphotogasm_lastUnicorn.safetensors')


  if Model_from_URL:
      for m in Model_from_URL.split(','):
        %cd {root}/stable-diffusion-webui/models/Stable-diffusion
        downloadModel(m)
        if Save_a_copy_in_Google_Drive and gMode == GDriveSaveMode.Models_only:
          %cd {output_path}/models
          downloadModel(m)

  # download VAEs
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d {root}/stable-diffusion-webui/models/VAE/ -o vae-ft-mse-840000-ema-pruned.ckpt

def installxformers():
  #!pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.16/xformers-0.0.16+814314d.d20230118-cp38-cp38-linux_x86_64.whl
  #%pip install --no-deps -q https://github.com/brian6091/xformers-wheels/releases/download/0.0.15.dev0%2B4c06c79/xformers-0.0.15.dev0+4c06c79.d20221205-cp38-cp38-linux_x86_64.whl
  %pip install -q xformers

def updatePython():
  !python --version > /content/pyversion
  with open('/content/pyversion', 'r') as file:
      if '3.10' in file.read():
        print('Already python 3.10. Skip install.')
        return

  #install python 3.10
  !apt-get update -y
  !apt-get install python3.10

  #change alternatives
  !rm /usr/local/bin/python
  !rm /usr/local/bin/pip
  !sudo apt-get install python3.10-distutils
  !sudo update-alternatives --install /usr/local/bin/python python /usr/bin/python3.10 2
  !wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py



import requests
import os

def downloadFile(url, folder_path):
    # Ensure Google Drive is mounted
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

    # Temporary path for downloading the zip file
    temp_zip_path = '/content/temp.zip'

    # Download the file and save it to the temporary path
    with open(temp_zip_path, 'wb') as file:
        response = requests.get(url)
        file.write(response.content)

    # Full path in Google Drive
    full_path = os.path.join('/content/drive/MyDrive', folder_path)
    if not os.path.exists(full_path):
        os.makedirs(full_path)

    # Extract the files to the specified folder
    import zipfile
    with zipfile.ZipFile(temp_zip_path, 'r') as zip_ref:
        zip_ref.extractall(full_path)

    # Delete the temporary zip file
    os.remove(temp_zip_path)


def initSaveGoogleDriveModelOnly():
  # Use config files in google drive
  # ui-config.json
  if not os.path.exists(output_path + '/ui-config.json'):
    print("Create new ui-config.json file.")
    !wget https://github.com/enternalsaga/stablediffusion_webui/raw/master/ui-config.json -O {output_path + '/ui-config.json'}
  !ln -s {output_path + '/ui-config.json'} {root}/stable-diffusion-webui/

  # Config.json
  if not os.path.exists(output_path + '/config.json'):
    print("Create new config.json file.")
    !wget https://github.com/enternalsaga/stablediffusion_webui/raw/master/config.json -O {output_path + '/config.json'}
  !ln -s {output_path + '/config.json'} {root}/stable-diffusion-webui/

  # styles.csv
  if not os.path.exists(output_path + '/styles.csv'):
    print("Create new styles.csv file.")
    !wget https://raw.githubusercontent.com/enternalsaga/stablediffusion_webui/master/styles.csv -O {output_path + '/styles.csv'}
  !ln -s {output_path + '/styles.csv'} {root}/stable-diffusion-webui/



  !ln -s {output_path}/outputs

  # embeddings folder in Google Drive
  if not os.path.exists(output_path + '/embeddings/'):
    downloadFile('https://huggingface.co/datasets/enternalsaga/models_basic/resolve/main/embeddings.zip','AIDVN_SD/embeddings')
  !rm -rf embeddings
  !ln -s {output_path}/embeddings

  # save parameter file in google drive
  if not os.path.exists(output_path + '/params.txt'):
    !touch {output_path + '/params.txt'}
  !ln -s {output_path}/params.txt

  # link all models in the models folder
  %cd {root}/stable-diffusion-webui/models/Stable-diffusion
  models_in_google_drive = glob.glob(output_path + '/models/*')
  print('Models in Google Drive: %s'%models_in_google_drive)
  for f in models_in_google_drive:
    !ln -s {f}

  # upscalers folder in Google Drive
  if not os.path.exists(output_path + '/ESRGAN/'):
    downloadFile('https://huggingface.co/datasets/enternalsaga/models_basic/resolve/main/upscalers.zip','AIDVN_SD/ESRGAN')
  !rm -rf ESRGAN
  !ln -s {output_path}/ESRGAN

  # use lora model folder in google drive
  if not os.path.exists(output_path + '/Lora/'):
    downloadFile('https://huggingface.co/datasets/enternalsaga/models_basic/resolve/main/loras.zip','AIDVN_SD/Lora')
  %cd {root}/stable-diffusion-webui/models
  !rm -rf Lora
  !ln -s {output_path}/Lora

  # use hypernetwork folder in google drive
  !mkdir -p {output_path}/hypernetworks
  %cd {root}/stable-diffusion-webui/models
  !rm -rf hypernetworks
  !ln -s {output_path}/hypernetworks

  # use VAE folder in google drive
  !mkdir -p {output_path}/VAE
  %cd {root}/stable-diffusion-webui/models
  !rm -rf VAE
  !ln -s {output_path}/VAE

  # use ControlNet folder in google drive
  !mkdir -p {output_path}/ControlNet
  %cd {root}/stable-diffusion-webui/models
  !rm -rf ControlNet
  !ln -s {output_path}/ControlNet




def installControlNet_essential():
  print("Installing ControlNet extension...")
  %cd {root}/stable-diffusion-webui/extensions
  !git clone https://github.com/Mikubill/sd-webui-controlnet.git
  %cd {root}/stable-diffusion-webui/extensions/sd-webui-controlnet
  !pip install -r requirements.txt

  %cd {root}/stable-diffusion-webui/extensions/sd-webui-controlnet/models
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart.pth')
  downloadModel('https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus_sd15.safetensors')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd.pth')


  print("ControlNet install completed.")

def installControlNet_extra():
  print("Installing ControlNet extension...")
  %cd {root}/stable-diffusion-webui/extensions/sd-webui-controlnet

  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime.pth')
  downloadModel('https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth')
  downloadModel('https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth')


  print('Install QR code models...')
  downloadModel('https://huggingface.co/monster-labs/control_v1p_sd15_qrcode_monster/resolve/main/control_v1p_sd15_qrcode_monster.safetensors')
  downloadModel('https://huggingface.co/monster-labs/control_v1p_sd15_qrcode_monster/resolve/main/v2/control_v1p_sd15_qrcode_monster_v2.safetensors')
  downloadModel('https://huggingface.co/Nacholmo/controlnet-qr-pattern-sdxl/resolve/main/automatic1111/control_v01u_sdxl_qrpattern.safetensors')


  print("Install ControlNet SDXL models...")
  downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_canny_full.safetensors')
  downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_depth_mid.safetensors')
  downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ip-adapter_xl.pth')
  downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_blur.safetensors')
  downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_blur_anime.safetensors')
  downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_scribble_anime.safetensors')
  downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_recolor_256lora.safetensors')
  downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_sketch_256lora.safetensors')
  downloadModel('https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_diffusers_xl_lineart.safetensors')

  print("ControlNet install completed.")

def installCivitaiBrowserPlus():
  !git clone https://github.com/BlafKing/sd-civitai-browser-plus {root}/stable-diffusion-webui/extensions/sd-civitai-browser-plus
  !apt-get install libvulkan1

def installRegionalPrompter():
  !git clone https://github.com/hako-mikan/sd-webui-regional-prompter {root}/stable-diffusion-webui/extensions/sd-webui-regional-prompter


def installUltimateSDUpscale():
  %cd {root}/stable-diffusion-webui/extensions
  !git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111





def applyGitFilemode():
  # git default file mode prevents checkout and fail
  print('Apply git filemode false')
  !cd {root}/stable-diffusion-webui/repositories/k-diffusion;git config core.filemode false
  !cd {root}/stable-diffusion-webui/repositories/stable-diffusion-stability-ai;git config core.filemode false
  !cd {root}/stable-diffusion-webui/repositories/taming-transformers;git config core.filemode false
  !cd {root}/stable-diffusion-webui/repositories/CodeFormer;git config core.filemode false
  !cd {root}/stable-diffusion-webui/repositories/BLIP;git config core.filemode false

def cloneRepositories():
  !git clone https://github.com/crowsonkb/k-diffusion.git {root}/stable-diffusion-webui/repositories/k-diffusion
  !git clone https://github.com/Stability-AI/stablediffusion.git {root}/stable-diffusion-webui/repositories/stable-diffusion-stability-ai
  !git clone https://github.com/CompVis/taming-transformers.git {root}/stable-diffusion-webui/repositories/taming-transformers
  !git clone https://github.com/sczhou/CodeFormer.git {root}/stable-diffusion-webui/repositories/CodeFormer
  !git clone https://github.com/salesforce/BLIP.git {root}/stable-diffusion-webui/repositories/BLIP


def installExtensionsFromURL(urls):
  %cd {root}/stable-diffusion-webui/extensions
  for url in urls.split(','):
    print("Cloning extension from URL: %s"%url)
    !git clone {url}

def lowRamPatch():
  print('Apply lowram patch')
  !sed -i 's/dict()))$/dict())).cuda()/g'  {root}/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py

def searchAndReplace(filePath, orignalStr, newStr):
  orignalStr = orignalStr.replace('/', '\/')
  newStr = newStr.replace('/', '\/')
  !sed -i 's/{orignalStr}/{newStr}/g' {filePath}


def deleteRepos():
  # delete repository directories in webui
  !rm -rf {root}/stable-diffusion-webui/repositories

updatePython()

!mkdir -p {root}
os.chdir(root)
!apt-get -y install -qq aria2
!pip install pyngrok
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui



if gMode == GDriveSaveMode.Everything:
  # delete existing repositories and reclone so the file mode fix can be applied
  # otherwise some will only be cloned in the final launch, causing some to fail to checkout.
  deleteRepos()
  cloneRepositories()
  applyGitFilemode()

# fix torch, torchvision version mismatch error
# !pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117

# A1111 first launch
%cd {root}/stable-diffusion-webui

#!git checkout -f a3ddf464a2ed24c999f67ddfef7969f8291567be
!git checkout -f {version}
!COMMANDLINE_ARGS="--exit"  python launch.py


if gMode == GDriveSaveMode.Models_only:
  initSaveGoogleDriveModelOnly()

download_models()

if ControlNet_essential:
  installControlNet_essential()

if ControlNet_full:
  installControlNet_essential()
  installControlNet_extra()

if Civitai_browser_plus:
  installCivitaiBrowserPlus()

if Regional_Prompter:
  installRegionalPrompter()

if Ultimate_SD_Upscale:
  installUltimateSDUpscale()






installExtensionsFromURL(Extensions_from_URL)

# downgrade httpx to avoid TypeError: AsyncConnectionPool.__init__() got an unexpected keyword argument 'socket_options'
!pip install httpx==0.24.1

# clear output
if Clear_Log:
  clear()




if  gMode != GDriveSaveMode.Everything:
   lowRamPatch()

%cd {root}/stable-diffusion-webui
args = f'--gradio-img2img-tool color-sketch --enable-insecure-extension-access --gradio-queue --share --disable-model-loading-ram-optimization --opt-sdp-no-mem-attention --no-download-sd-model --esrgan-models-path /content/drive/MyDrive/AIDVN_StableDiffusion/ESRGAN'


print(f'WEBUI ARGUMENTS: {args}')
!python {root}/stable-diffusion-webui/launch.py {args}